In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if False: # if True: turn off GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    gpu = False
else:
    gpu = True

from tensorflow import keras
from copy import deepcopy
import tensorflow as tf
from glob import glob
import numpy as np
import random


from utils.cityscapesSequence import CitySequence, labels
import utils.dataUtils as dataUtils
from modelclass import DeeplabV3

if gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 8))])

print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.1


In [2]:
DATA_DIR = os.path.join("..", "cityscapes", "allData")

batchSize = 8
imageSize = 256
lrs = [5e-4, 1e-4, 5e-5, 1e-5, 5e-6]
alphas = [1., 0.9, 0.8, 0.7, 0.6]


##### Dataset


In [3]:
def getData(path="", type="gtFine", extra=False, nSplits=5, seed=0):
    # Load data
    xTrainVal, yTrainVal, xTest, yTest = dataUtils.getXY(path=path, extra=extra, type=type)
    print("TrainVal: ", len(xTrainVal), "Test: ", len(yTest))

    # Split the data in train and validation folds
    return dataUtils.splitTrainValFolds(xTrainVal, yTrainVal, nSplits=nSplits, seed=seed)


def createDatasetObject(trainValFolds, imageSize=imageSize, batchSize=1, remap="binary"):
    for i, (xTrain, yTrain, xVal, yVal) in enumerate(trainValFolds):
        print("Fold", i, "Train: ", len(xTrain), "Val: ", len(xVal))

        dataUtils.dataShuffle(xData=xTrain, yData=yTrain)

        # Create the dataset
        trainDataset = CitySequence(xTrain, yTrain, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=True, verticalFlip=True, brightness=0.1)
        valDataset = CitySequence(xVal, yVal, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=False, verticalFlip=False, brightness=0)

        yield trainDataset, valDataset, trainDataset.nClasses


kf = getData(path=DATA_DIR, type="gtFine", extra=False, nSplits=5, seed=13)
data = createDatasetObject(kf, imageSize=imageSize, batchSize=batchSize, remap="binary")

Getting data from: ..\cityscapes\allData
Getting train data from: ..\cityscapes\allData\leftImg8bit\train\*\*_leftImg8bit.png
Getting val data from: ..\cityscapes\allData\leftImg8bit\val\*\*_leftImg8bit.png
Getting test data from: ..\cityscapes\allData\leftImg8bit\test\*\*_leftImg8bit.png
TrainVal:  3475 Test:  1525


##### Train

In [4]:
losses = {}

for i, (trainDataset, valDataset, nClasses) in enumerate(data):
    for lr in lrs:
        for alpha in alphas:
            print("Fold:", i, "learning rate:", lr, "alpha:", alpha)
            
            MODEL_NAME = "deeplabv3plus_fold_" + str(i) + "_lr_" + str(lr) + "_alpha_" + str(alpha) + "_binary" + str(imageSize)

            model = DeeplabV3(imageSize=imageSize, nClasses=nClasses, alpha=alpha, modelName=MODEL_NAME)
            
            model.run(trainDataset, valDataset, epochs=1000, learningRate=lr, batchSize=batchSize, monitor="val_loss", log=True)

            # Evaluate the model
            losses[MODEL_NAME] = model.evaluate(valDataset)
            print("losses", losses)

Fold 0 Train:  2780 Val:  695
Fold: 0 learning rate: 0.0005 alpha: 1.0
Epoch 1/1000
348/348 [==============================] - ETA: 0s - loss: 57.3811 - diceAccuracy: 0.9060 - jaccardDistance: 1.6425
Epoch 1: val_loss improved from inf to 8.45420, saving model to model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


348/348 [==============================] - 402s 1s/step - loss: 57.3811 - diceAccuracy: 0.9060 - jaccardDistance: 1.6425 - val_loss: 8.4542 - val_diceAccuracy: 0.5075 - val_jaccardDistance: 6.5977 - lr: 5.0000e-04
Epoch 2/1000
348/348 [==============================] - ETA: 0s - loss: 2.2428 - diceAccuracy: 0.9298 - jaccardDistance: 1.2837
Epoch 2: val_loss improved from 8.45420 to 0.47994, saving model to model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


...

INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


348/348 [==============================] - 376s 1s/step - loss: 0.2137 - diceAccuracy: 0.9441 - jaccardDistance: 1.0452 - val_loss: 0.2075 - val_diceAccuracy: 0.9376 - val_jaccardDistance: 1.1499 - lr: 5.0000e-04
Epoch 27/1000
348/348 [==============================] - ETA: 0s - loss: 0.4245 - diceAccuracy: 0.9428 - jaccardDistance: 1.0631
Epoch 27: val_loss did not improve from 0.20749
348/348 [==============================] - 344s 990ms/step - loss: 0.4245 - diceAccuracy: 0.9428 - jaccardDistance: 1.0631 - val_loss: 0.5607 - val_diceAccuracy: 0.4407 - val_jaccardDistance: 7.1445 - lr: 5.0000e-04
Epoch 28/1000
348/348 [==============================] - ETA: 0s - loss: 0.2547 - diceAccuracy: 0.9453 - jaccardDistance: 1.0252
Epoch 28: val_loss did not improve from 0.20749
348/348 [==============================] - 348s 1s/step - loss: 0.2547 - diceAccuracy: 0.9453 - jaccardDistance: 1.0252 - val_loss: 0.5058 - val_diceAccuracy: 0.4419 - val_jaccardDistance: 7.1354 - lr: 5.0000e-04
Epoc

INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


348/348 [==============================] - 375s 1s/step - loss: 0.1972 - diceAccuracy: 0.9459 - jaccardDistance: 1.0142 - val_loss: 0.2028 - val_diceAccuracy: 0.9199 - val_jaccardDistance: 1.4703 - lr: 5.0000e-04
Epoch 37/1000
348/348 [==============================] - ETA: 0s - loss: 0.1888 - diceAccuracy: 0.9464 - jaccardDistance: 1.0045
Epoch 37: val_loss did not improve from 0.20285
348/348 [==============================] - 344s 989ms/step - loss: 0.1888 - diceAccuracy: 0.9464 - jaccardDistance: 1.0045 - val_loss: 0.2080 - val_diceAccuracy: 0.9162 - val_jaccardDistance: 1.5175 - lr: 5.0000e-04
Epoch 38/1000
280/348 [=======================>......] - ETA: 55s - loss: 0.1849 - diceAccuracy: 0.9462 - jaccardDistance: 1.0079

In [ ]:
losses

{}

##### TFLITE Converter


In [ ]:
from utils.imageUtils import representativeDatasetGen

representativeData = representativeDatasetGen(path="../cityscapes/alldata")

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData, supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS])

MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData)

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)

model.load_weights(MODEL_PATH)

